<a href="https://colab.research.google.com/github/mohmiim/MLIntroduction/blob/master/session-5/Session_5_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc1


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import xception, inception_v3, resnet_v2, vgg19,densenet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout, Conv2D, Flatten, MaxPool2D
from tensorflow.keras.activations import relu,softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [0]:

# https://github.com/mohmiim/MLIntroduction/raw/master/data/data.zip
# https://github.com/mohmiim/MLIntroduction/raw/master/data/smallData.zi


In [0]:
import os

!wget --no-check-certificate \
    https://github.com/mohmiim/MLIntroduction/raw/master/data/data.zip \
    -O /tmp/data.zip

In [0]:
import zipfile
def unzip(file,target):
  zip_ref = zipfile.ZipFile(file, 'r')
  zip_ref.extractall(target)
  zip_ref.close()

unzip("/tmp/data.zip","/tmp")

In [5]:

print(os.listdir("/tmp/data"))


['test', 'train', '.gitignore']


In [0]:
#default sizes
Image_Width = 100
Image_Height = 100
Image_Depth = 3
targetSize = (Image_Width,Image_Height)
targetSize_withdepth = (Image_Width,Image_Height,Image_Depth)

epochs = 100
x_train = []
y_train = []
y_labels = []

In [7]:
#define the sub folders for both training and test
training = os.path.join("/tmp/data",'train')

train_data_generator = ImageDataGenerator(preprocessing_function=xception.preprocess_input,
                                          width_shift_range=0.2,
                                          height_shift_range=0.2,
                                          zoom_range=0.2,
                                          fill_mode='nearest')


train_generator = train_data_generator.flow_from_directory(training,
                                                           batch_size=229,
                                                           target_size=targetSize,
                                                           #seed=12
                                                           shuffle=False
                                                           )

Found 229 images belonging to 5 classes.


In [8]:
y_train =  train_generator.classes
for k in train_generator.class_indices.keys():
    y_labels.append(k)
y_train = to_categorical(y_train)

print(len(y_train))

229


In [0]:
# NOW WE LOAD THE PRE_TRAINED MODEL
FEATURE_EXTRACTOR = vgg19.VGG19(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model = Sequential()
model.add(FEATURE_EXTRACTOR)
model.add(Flatten())
features_x = model.predict(train_generator, steps=1)
print(features_x.shape)

In [0]:
FEATURE_EXTRACTOR1 = xception.Xception(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model1 = Sequential()
model1.add(FEATURE_EXTRACTOR1)
model1.add(Flatten())
features_x1 = model1.predict(train_generator, steps=1)
print(features_x1.shape)

In [0]:
FEATURE_EXTRACTOR2 = resnet_v2.ResNet152V2(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model2 = Sequential()
model2.add(FEATURE_EXTRACTOR2)
model2.add(Flatten())
features_x2 = model2.predict(train_generator, steps=1)
print(features_x2.shape)

In [0]:
FEATURE_EXTRACTOR3 = inception_v3.InceptionV3(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model3 = Sequential()
model3.add(FEATURE_EXTRACTOR3)
model3.add(Flatten())
features_x3 = model3.predict(train_generator, steps=1)
print(features_x3.shape)

In [0]:
FEATURE_EXTRACTOR4 = densenet.DenseNet201(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model4 = Sequential()
model4.add(FEATURE_EXTRACTOR4)
model4.add(Flatten())
features_x4 = model4.predict(train_generator,steps=1)
print(features_x4.shape)

In [14]:
import numpy as np
all_features = np.concatenate((features_x, features_x1,features_x2,features_x3,features_x4), axis=1)
print(all_features.shape)

(229, 75136)


In [15]:
model = Sequential()

#add our layers
model.add(Flatten(input_shape=all_features.shape[1:]))
model.add(Dense(128,activation=relu))
model.add(Dropout(0.1))
model.add(Dense(64,activation=relu))
model.add(Dense(len(y_labels),activation='softmax'))
history = model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=['accuracy'])

model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 75136)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               9617536   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
Total params: 9,626,117
Trainable params: 9,626,117
Non-trainable params: 0
_________________________________________________________________


In [0]:
from tensorflow.keras.callbacks import Callback
class myCallBacks(Callback):
    
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('loss')<=self.loss) :
            print("\n Reached {1} loss on epoch {0}, stopping training".format(epoch+1,self.loss))
            self.model.stop_training = True
    
    def __init__(self, loss=1E-4):
        self.loss = loss

In [0]:
epochs = 500
callBack = myCallBacks(loss=1E-7)
model.fit(all_features,y_train,epochs=epochs,shuffle=True,verbose=2,callbacks=[callBack])

In [18]:
test_data_generator = ImageDataGenerator(preprocessing_function=xception.preprocess_input)
test_generator = test_data_generator.flow_from_directory("/tmp/data/test",
                                                         target_size=(100,100),
                                                         batch_size=93,
                                                         shuffle=False)

Found 93 images belonging to 5 classes.


In [0]:
FEATURE_EXTRACTOR =  vgg19.VGG19(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model0 = Sequential()
model0.add(FEATURE_EXTRACTOR)
model0.add(Flatten())
features_x = model0.predict(test_generator,steps=1)
print(type(features_x).__name__)
print(features_x.shape)

FEATURE_EXTRACTOR1 = xception.Xception(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model1 = Sequential()
model1.add(FEATURE_EXTRACTOR1)
model1.add(Flatten())
features_x1 = model1.predict(test_generator,steps=1)
print(type(features_x1).__name__)
print(features_x1.shape)

FEATURE_EXTRACTOR2 = resnet_v2.ResNet152V2(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model2 = Sequential()
model2.add(FEATURE_EXTRACTOR2)
model2.add(Flatten())
features_x2 = model2.predict(test_generator, steps=1)
print(type(features_x2).__name__)
print(features_x2.shape)

FEATURE_EXTRACTOR3 = inception_v3.InceptionV3(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model3 = Sequential()
model3.add(FEATURE_EXTRACTOR3)
model3.add(Flatten())
features_x3 = model3.predict(test_generator,steps=1)
print(type(features_x3).__name__)
print(features_x3.shape)

FEATURE_EXTRACTOR4 = densenet.DenseNet201(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)
model4 = Sequential()
model4.add(FEATURE_EXTRACTOR4)
model4.add(Flatten())
features_x4 = model4.predict(test_generator,steps=1)
print(type(features_x4).__name__)
print(features_x4.shape)

all_features = np.concatenate((features_x, features_x1,features_x2,features_x3,features_x4), axis=1)


In [0]:
predictions = model.predict(all_features)

In [0]:
from sklearn.metrics import confusion_matrix,classification_report
row_index = predictions.argmax(axis=1)
filenames = test_generator.filenames
nb_samples = len(filenames)
y_true = test_generator.classes
target_names = test_generator.class_indices.keys()
print(target_names)
print(confusion_matrix(y_true, row_index))
print('Classification Report')
target_names = test_generator.class_indices.keys()
print(classification_report(test_generator.classes, row_index, target_names=target_names))